### is_us_market_open_now

In [ ]:
import time
from datetime import datetime, timedelta
import pytz
import exchange_calendars as ec
import argparse
# --------------- 거래소 캘린더 및 더미 API 함수 ---------------
# NYSE 캘린더 객체 생성
XNYS = ec.get_calendar("XNYS")
def is_us_market_open_now(check_time=None):
    """
    주어진 시각(미국 동부 기준)이 정규장 중인지 반환.
    (휴장일, 주말, 조기폐장 자동 반영)
    """
    if check_time is None:
        now_et = datetime.now(pytz.timezone("America/New_York"))
    else:
        if check_time.tzinfo is None:
            check_time = check_time.replace(tzinfo=pytz.timezone("America/New_York"))
        now_et = check_time.astimezone(pytz.timezone("America/New_York"))
    date_str = now_et.strftime('%Y-%m-%d')
    if date_str not in XNYS.sessions:
        return False
    market_open_utc = XNYS.opens.loc[date_str]
    market_close_utc = XNYS.closes.loc[date_str]
    now_utc = now_et.astimezone(pytz.utc)
    return market_open_utc <= now_utc <= market_close_utc
def get_remaining_market_time(check_time=None):
    """
    현재 정규장이 열려 있다고 가정하고, 장 마감까지 남은 시간(timedelta)을 계산.
    장이 이미 종료되었으면 None 반환.
    """
    if check_time is None:
        now_et = datetime.now(pytz.timezone("America/New_York"))
    else:
        if check_time.tzinfo is None:
            check_time = check_time.replace(tzinfo=pytz.timezone("America/New_York"))
        now_et = check_time.astimezone(pytz.timezone("America/New_York"))
    date_str = now_et.strftime('%Y-%m-%d')
    if date_str not in XNYS.sessions:
        return None
    market_close_utc = XNYS.closes.loc[date_str]
    now_utc = now_et.astimezone(pytz.utc)
    remaining_time = market_close_utc - now_utc
    return remaining_time if remaining_time.total_seconds() > 0 else None
def get_time_until_next_market_open(check_time=None):
    """
    현재 장이 종료된 경우, 다음 거래일 개장까지 남은 시간(timedelta)을 계산.
    현재 장이 아직 열려 있다면 None 반환.
    """
    if check_time is None:
        now_et = datetime.now(pytz.timezone("America/New_York"))
    else:
        if check_time.tzinfo is None:
            check_time = check_time.replace(tzinfo=pytz.timezone("America/New_York"))
        now_et = check_time.astimezone(pytz.timezone("America/New_York"))
    date_str = now_et.strftime('%Y-%m-%d')
    if date_str in XNYS.sessions:
        market_close_utc = XNYS.closes.loc[date_str]
        now_utc = now_et.astimezone(pytz.utc)
        if now_utc < market_close_utc:
            return None  # 아직 장중이면 다음 개장 시간 계산 안 함.
    # 다음 거래일 찾기
    next_session = XNYS.sessions[XNYS.sessions > date_str].min()
    next_market_open_utc = XNYS.opens.loc[next_session]
    now_utc = now_et.astimezone(pytz.utc)
    return next_market_open_utc - now_utc
def get_present_stock_price(stock_name):
    # 예: 현재 가격을 반환 (실제 API 호출)
    return 100.0
def get_average_my_stock_price(stock_name):
    # 예: 내 평균 매수가격 (실제 계좌 정보)
    return 95.0
def all_sell(stock_name=None):
    # 예: 전량 매도 주문 실행 (실제 API 호출)
    print(">> ALL SELL 주문 실행")
    return "ALL_SELL_ORDER_1"
def half_order_info(order_no):
    # 예: 0.5 회차 주문 정보 확인 (실제 API 호출)
    # key "체결여부"로 체결 상태 반환
    return {"order_price": 98.0, "체결여부": True}
def loc_order_info(order_no):
    # 예: LOC 주문 정보 확인
    return {"order_price": 102.0, "체결여부": True}
def condition_order_avg(stock_name, price):
    # 예: 해당 가격 조건으로 주문 걸기 (실제 API 호출)
    order_no = f"ORDER_{price}"
    print(f">> 조건 주문({order_no}) 생성")
    return order_no
def cancel_order(order_no):
    # 예: 주문 취소 (실제 API 호출)
    print(f">> 주문 {order_no} 취소됨")
# --------------- 메인 트레이딩 루프 ---------------
def main_trading_loop(stock_name="SOXL", split_no=40):
    """
    메인 트레이딩 루프.
    :param stock_name: 거래 종목 (티커)
    :param split_no: 분할 매수 횟수 (기본 40)
    """
    used_split = 0         # 사용한 분할 매수 회차
    present_price = get_present_stock_price(stock_name)
    reservoir = split_no * 2 * present_price  # 예: 40 * 2 * 현재가격
    my_stocks = 0

    all_sell_order_no = None
    half_order_active = False
    half_order_no = None 
    half_success = False 
    loc_order_active = False 
    loc_order_no = None 
    loc_success = False 
    out_of_amount = False
    terminate = False

    print(">> 트레이딩 루프 시작")
    while True:
        try:
            time.sleep(3)  # 주기적 확인
            if not is_us_market_open_now():
                # 장이 열리지 않으면 다음 개장까지 대기
                next_open = get_time_until_next_market_open()
                if next_open:
                    sleep_seconds = max(next_open.total_seconds(), 0)
                    print(f"장 마감. 다음 개장까지 {sleep_seconds/60:.1f}분 대기합니다.")
                    time.sleep(sleep_seconds)
                continue

            # 장이 열렸을 때 내부 루프 실행
            while is_us_market_open_now():
                now_et = datetime.now(pytz.timezone("America/New_York"))
                remain_time = get_remaining_market_time(now_et)
                if remain_time is None:
                    # 장 종료됨
                    terminate = True
                    print(">> 정규장 종료 감지")
                    break

                # 현재 주가 및 내 평단가 조회
                present_stock_price = get_present_stock_price(stock_name)
                average_my_stock_price = get_average_my_stock_price(stock_name)

                # 남은 예산 계산 (단순 예시)
                half_cost = half_order_info(half_order_no)["order_price"] if (half_order_active and half_order_no) else 0
                loc_cost = loc_order_info(loc_order_no)["order_price"] if (loc_order_active and loc_order_no) else 0
                remain_reservoir = reservoir - (my_stocks * average_my_stock_price) - half_cost - loc_cost
                print(f">> 남은 예산: {remain_reservoir:.2f}")

                # 매도 조건: 평단가의 10% 이상 상승 시 전량 매도
                if average_my_stock_price > 0 and present_stock_price >= average_my_stock_price * 1.1:
                    if all_sell_order_no is None:
                        all_sell_order_no = all_sell(stock_name)
                        print(">> 목표 수익 도달: 전량 매도 실행")
                
                # 원금 소진 조건
                if used_split < split_no * 2:
                    if remain_reservoir < present_stock_price:
                        if present_stock_price >= average_my_stock_price * 0.9:
                            if all_sell_order_no is None:
                                all_sell_order_no = all_sell(stock_name)
                                print(">> 원금 소진 조건 충족: 전량 매도 실행")
                        else:
                            out_of_amount = True
                            print(">> 잔고 부족: 추가 매수 불가")
                            break

                # 0.5 회차 매수 주문 (장중 가격이 평단가 이하 혹은 첫 매수)
                if not half_order_active:
                    half_order_no = condition_order_avg(stock_name, average_my_stock_price)
                    half_order_active = True
                    print(f">> 0.5회차 주문 생성: {half_order_no}")
                elif half_order_active and not half_success:
                    info = half_order_info(half_order_no)
                    if info.get("체결여부"):
                        half_success = True
                        used_split += 1
                        print(">> 0.5회차 주문 체결됨")
                        # 실제 체결 시 내 평단가 및 보유 주식 업데이트 필요
                    elif not info.get("체결여부") and loc_order_active:
                        cancel_order(half_order_no)
                        print(">> 0.5회차 주문 미체결 -> 취소 처리")

                # LOC 주문 (잔여 시간이 10분 이하인 경우)
                if remain_time is not None and remain_time <= timedelta(minutes=10):
                    if not loc_order_active:
                        # 조건에 따라 주문 가격 결정
                        if present_stock_price <= average_my_stock_price * 1.15:
                            order_price = present_stock_price
                        else:
                            order_price = average_my_stock_price * 1.15
                        loc_order_no = condition_order_avg(stock_name, order_price)
                        loc_order_active = True
                        print(f">> LOC 주문 생성: {loc_order_no}")
                    elif loc_order_active and not loc_success:
                        info = loc_order_info(loc_order_no)
                        if info.get("체결여부"):
                            loc_success = True
                            used_split += 1
                            print(">> LOC 주문 체결됨")
                time.sleep(3)  # 내부 루프 주기
            # 내부 while 종료 후: 장 종료 또는 잔고 부족 등
            if terminate:
                if not loc_success and loc_order_no is not None:
                    cancel_order(loc_order_no)
                    print(">> LOC 주문 취소 (장 종료 전)")
                if all_sell_order_no is not None:
                    cancel_order(all_sell_order_no)
                    print(">> 전량 매도 주문 취소")
                print(">> 오늘 거래 종료, 로그 기록 후 재시작 준비")
                # 로그 파일 생성 및 후처리 가능
                next_open = get_time_until_next_market_open()
                if next_open:
                    # 다음 개장 10분 전까지 대기 (sleep 시간 계산)
                    sleep_seconds = max(next_open.total_seconds() - 600, 0)
                    print(f">> 다음 거래일까지 {sleep_seconds/60:.1f}분 대기")
                    time.sleep(sleep_seconds)
                # 거래일 재시작을 위해 변수 초기화
                terminate = False
                all_sell_order_no = None 
                half_order_active = False
                half_order_no = None 
                half_success = False 
                loc_order_active = False
                loc_order_no = None 
                loc_success = False 
            if out_of_amount:
                print(">> 잔고 부족 경고: 사용자 알림 후 종료")
                # 실제 환경에서는 이메일이나 HTTPS 통신으로 경고 전송
                break
        except Exception as e:
            print(f">> 에러 발생: {e}")
            time.sleep(5)  # 에러 발생 시 잠시 대기 후 재시작

if __name__ == "__main__":
    # argparse를 활용해 시스템 전달 인수로 초깃값 설정
    parser = argparse.ArgumentParser(description="라오어 무한매수법 트레이딩 봇")
    parser.add_argument("--stock", type=str, default="SOXL", help="거래 종목 (티커)")
    parser.add_argument("--splits", type=int, default=40, help="분할 매수 횟수 (기본: 40)")
    args = parser.parse_args()

    print(f">> 초깃값: 종목={args.stock}, 분할 횟수={args.splits}")
    main_trading_loop(stock_name=args.stock, split_no=args.splits)

>> 트레이딩 루프 시작
장 마감. 다음 개장까지 631.5분 대기합니다.


In [ ]:
import exchange_calendars as ec
from datetime import datetime, timedelta
import pytz

# NYSE 캘린더 객체 생성
XNYS = ec.get_calendar("XNYS")

def is_us_market_open_now(check_time=None):
    """
    주어진 시각(미국 동부 기준)을 UTC로 변환 후, 현재 시장이 열려있는지 여부를 반환.
    (휴장일, 주말, 조기폐장 자동 반영)
    """
    # (A) 확인 시각이 없으면 현재 미국 동부시간(ET) 사용
    if check_time is None:
        now_et = datetime.now(pytz.timezone("America/New_York"))  # 현재 ET
    else:
        if check_time.tzinfo is None:
            check_time = check_time.replace(tzinfo=pytz.timezone("America/New_York"))
        now_et = check_time.astimezone(pytz.timezone("America/New_York"))  # ET 변환

    # (B) ET → UTC 변환
    now_utc = now_et.astimezone(pytz.utc)  

    print("🕒 현재 미국 동부시간 (ET):", now_et)
    print("🌍 변환된 현재 UTC 시간:", now_utc)

    # (C) 오늘 날짜를 UTC 기준으로 가져오기
    date_str = now_utc.strftime('%Y-%m-%d')

    # (D) 오늘이 거래 가능한 날인지 확인
    if date_str not in XNYS.sessions:
        print("🚫 오늘은 거래일이 아닙니다 (휴장)")
        return False

    # (E) 오늘의 개장 및 폐장 시간 (UTC)
    market_open_utc = XNYS.opens.loc[date_str]
    market_close_utc = XNYS.closes.loc[date_str]

    # (F) 로그 출력 (디버깅용)
    print(f"📅 거래일 (UTC): {date_str}")
    print(f"🕒 개장 시간 (UTC): {market_open_utc}")
    print(f"🕒 폐장 시간 (UTC): {market_close_utc}")
    print(f"🕒 현재 시간 (UTC): {now_utc}")

    # (G) 현재 시간이 개장 ~ 폐장 사이에 있는지 확인
    return market_open_utc <= now_utc <= market_close_utc
def get_remaining_market_time(check_time=None):
    """
    현재 정규장이 열려 있다고 가정하고, 장 마감까지 남은 시간(timedelta)을 계산.
    장이 이미 종료되었으면 None 반환.
    """
    if check_time is None:
        now_et = datetime.now(pytz.timezone("America/New_York"))
    else:
        if check_time.tzinfo is None:
            check_time = check_time.replace(tzinfo=pytz.timezone("America/New_York"))
        now_et = check_time.astimezone(pytz.timezone("America/New_York"))

    now_utc = now_et.astimezone(pytz.utc)  
    date_str = now_utc.strftime('%Y-%m-%d')

    if date_str not in XNYS.sessions:
        return None

    market_close_utc = XNYS.closes.loc[date_str]
    remaining_time = market_close_utc - now_utc

    return remaining_time if remaining_time.total_seconds() > 0 else None

def get_time_until_next_market_open(check_time=None):
    plus =  timedelta(hours= 8, minutes=0) 
    """
    현재 장이 종료된 경우, 다음 거래일 개장까지 남은 시간(timedelta)을 계산.
    현재 장이 아직 열려 있다면 None 반환.
    """
    if check_time is None:
        now_et = datetime.now(pytz.timezone("America/New_York"))
    else:
        if check_time.tzinfo is None:
            check_time = check_time.replace(tzinfo=pytz.timezone("America/New_York"))
        now_et = check_time.astimezone(pytz.timezone("America/New_York"))

    now_utc = now_et.astimezone(pytz.utc)   - plus
    date_str = now_utc.strftime('%Y-%m-%d')
    print(1, now_utc)
    if date_str in XNYS.sessions:
        market_open_utc = XNYS.opens.loc[date_str]
        market_close_utc = XNYS.closes.loc[date_str]
        print(2, market_open_utc)
        print(3, market_close_utc)
        # 현재 시간이 장 시작 전이면, 오늘 개장까지 남은 시간 계산
        if now_utc < market_open_utc:
            return market_open_utc - now_utc
        
        # 현재 시간이 장 마감 이후이면, 다음 거래일 개장까지 남은 시간 계산
        if now_utc > market_close_utc:
            next_session = XNYS.sessions[XNYS.sessions > date_str].min()
            next_market_open_utc = XNYS.opens.loc[next_session]
            return next_market_open_utc - now_utc

    # 오늘이 휴장일이면 다음 거래일 찾기
    next_session = XNYS.sessions[XNYS.sessions > date_str].min()
    next_market_open_utc = XNYS.opens.loc[next_session]
    return next_market_open_utc - now_utc

# ✅ 실행 예제
if __name__ == "__main__":
    market_open = is_us_market_open_now()
    print("✅ 시장 개장 여부:", "✅ 열림" if market_open else "❌ 닫힘")


🕒 현재 미국 동부시간 (ET): 2025-03-06 23:24:23.036143-05:00
🌍 변환된 현재 UTC 시간: 2025-03-07 04:24:23.036143+00:00
📅 거래일 (UTC): 2025-03-07
🕒 개장 시간 (UTC): 2025-03-07 14:30:00+00:00
🕒 폐장 시간 (UTC): 2025-03-07 21:00:00+00:00
🕒 현재 시간 (UTC): 2025-03-07 04:24:23.036143+00:00
✅ 시장 개장 여부: ❌ 닫힘


In [30]:
from datetime import datetime, timedelta
import pytz
import exchange_calendars as ec

# NYSE 캘린더 객체
XNYS = ec.get_calendar("XNYS")

def get_remaining_market_time(check_time=None):
    """
    현재 정규장이 열려 있다고 가정하고, 장 마감까지 남은 시간(timedelta)을 계산.
    장이 이미 종료되었으면 None 반환.
    """
    if check_time is None:
        now_et = datetime.now(pytz.timezone("America/New_York"))
    else:
        if check_time.tzinfo is None:
            check_time = check_time.replace(tzinfo=pytz.timezone("America/New_York"))
        now_et = check_time.astimezone(pytz.timezone("America/New_York"))

    now_utc = now_et.astimezone(pytz.utc)  
    date_str = now_utc.strftime('%Y-%m-%d')

    if date_str not in XNYS.sessions:
        return None

    market_close_utc = XNYS.closes.loc[date_str]
    remaining_time = market_close_utc - now_utc

    return remaining_time if remaining_time.total_seconds() > 0 else None

def get_time_until_next_market_open(check_time=None):
    plus =  timedelta(hours= 8, minutes=0) 
    """
    현재 장이 종료된 경우, 다음 거래일 개장까지 남은 시간(timedelta)을 계산.
    현재 장이 아직 열려 있다면 None 반환.
    """
    if check_time is None:
        now_et = datetime.now(pytz.timezone("America/New_York"))
    else:
        if check_time.tzinfo is None:
            check_time = check_time.replace(tzinfo=pytz.timezone("America/New_York"))
        now_et = check_time.astimezone(pytz.timezone("America/New_York"))

    now_utc = now_et.astimezone(pytz.utc)   - plus
    date_str = now_utc.strftime('%Y-%m-%d')
    print(1, now_utc)
    if date_str in XNYS.sessions:
        market_open_utc = XNYS.opens.loc[date_str]
        market_close_utc = XNYS.closes.loc[date_str]
        print(2, market_open_utc)
        print(3, market_close_utc)
        # 현재 시간이 장 시작 전이면, 오늘 개장까지 남은 시간 계산
        if now_utc < market_open_utc:
            return market_open_utc - now_utc
        
        # 현재 시간이 장 마감 이후이면, 다음 거래일 개장까지 남은 시간 계산
        if now_utc > market_close_utc:
            next_session = XNYS.sessions[XNYS.sessions > date_str].min()
            next_market_open_utc = XNYS.opens.loc[next_session]
            return next_market_open_utc - now_utc

    # 오늘이 휴장일이면 다음 거래일 찾기
    next_session = XNYS.sessions[XNYS.sessions > date_str].min()
    next_market_open_utc = XNYS.opens.loc[next_session]
    return next_market_open_utc - now_utc

print(get_remaining_market_time() )
print(get_time_until_next_market_open())

0 days 15:51:07.679082
1 2025-03-06 21:08:52.321919+00:00
2 2025-03-06 14:30:00+00:00
3 2025-03-06 21:00:00+00:00
0 days 17:21:07.678081


In [39]:
import yfinance as yf

# TSLL의 1분봉 데이터 가져오기 (기간: 최근 1일)
df = yf.download("TSLL", period="1d", interval="1m")  # 1분 간격 데이터
print(df.head())

# CSV로 저장
df.to_csv("TSLL_1min.csv")
df


[*********************100%***********************]  1 of 1 completed

Price                        Close     High      Low   Open    Volume
Ticker                        TSLL     TSLL     TSLL   TSLL      TSLL
Datetime                                                             
2025-03-06 14:30:00+00:00  11.2499  11.5000  11.1712  11.48  16327701
2025-03-06 14:31:00+00:00  11.2150  11.3200  11.1750  11.23    580701
2025-03-06 14:32:00+00:00  11.0899  11.2700  11.0650  11.22    770561
2025-03-06 14:33:00+00:00  11.1400  11.1799  11.0400  11.08    623511
2025-03-06 14:34:00+00:00  11.1200  11.1500  11.0600  11.14    647520


Price,Close,High,Low,Open,Volume
Ticker,TSLL,TSLL,TSLL,TSLL,TSLL
Datetime,,,,,
2025-03-06 14:30:00+00:00,11.2499,11.5000,11.1712,11.4800,16327701
2025-03-06 14:31:00+00:00,11.2150,11.3200,11.1750,11.2300,580701
2025-03-06 14:32:00+00:00,11.0899,11.2700,11.0650,11.2200,770561
2025-03-06 14:33:00+00:00,11.1400,11.1799,11.0400,11.0800,623511
2025-03-06 14:34:00+00:00,11.1200,11.1500,11.0600,11.1400,647520
...,...,...,...,...,...
2025-03-06 20:55:00+00:00,10.7599,10.8300,10.7150,10.7400,560008
2025-03-06 20:56:00+00:00,10.7150,10.7654,10.7150,10.7654,188699


In [43]:
import yfinance as yf

# 특정 날짜 범위의 데이터 다운로드
start_date = "2024-01-02"
end_date = "2024-01-03"  # 하루 추가
df = yf.download("TSLL", start=start_date, end=end_date, interval="1m")
# 데이터 출력 및 저장
# df.to_csv("TSLL_daily_data.csv")
print(df.head())

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['TSLL']: YFPricesMissingError('possibly delisted; no price data found  (1m 2024-01-02 -> 2024-01-03) (Yahoo error = "1m data not available for startTime=1704171600 and endTime=1704258000. The requested range must be within the last 30 days.")')


Empty DataFrame
Columns: [(Adj Close, TSLL), (Close, TSLL), (High, TSLL), (Low, TSLL), (Open, TSLL), (Volume, TSLL)]
Index: []
